In [64]:
import requests
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings('ignore')

In [65]:
def getHUC(lat, lon):
    """
    Function to get the HUC12 - HUC2 ids for a given latitude and longitude.
    Latitude and Longitude should be in WGS84 (EPSG:4326)
    """

    # the url of the web service for WBD info
    root='https://arcgis.cuahsi.org/arcgis/services/US_WBD/HUC_WBD/MapServer/WFSServer'

    # assumes that the input coords are in WGS84, i.e. EPSG:4326
    params = {'service': 'WFS',
              'request': 'GetFeature',
              'bbox': f"{lon}, {lat}, {lon}, {lat}",         
              'typeName': 'HUC_WBD:HUC12_US',
              'SrsName': 'EPSG:4326'}

    # make the WFS request
    r = requests.get(url = root, params = params, verify=False)
    
    # exit if we don't get a 200 response
    if r.status_code != 200:
        print(f'Error getting HUC for point: {lat}, {lon}')
        return None
    
    # load the response xml into memory
    root_xml = root = ET.fromstring(r.text)
    
    # define xml namespaces to make parsing easier
    ns = {'xs': "http://www.w3.org/2001/XMLSchema",
          'wfs':"http://www.opengis.net/wfs/2.0",
          'gml':"http://www.opengis.net/gml/3.2",
          'US_WBD_HUC_WBD':"http://arcgis.cuahsi.org/arcgis/services/US_WBD/HUC_WBD/MapServer/WFSServer",
          'xlink':"http://www.w3.org/1999/xlink",
          'xsi': "http://www.w3.org/2001/XMLSchema-instance" 
         }
    
    # get the wfs:member element
    hucs = root_xml.findall('wfs:member', ns)
    
    # we should only get a single HUC since we're querying a point location
    huc12 = root_xml.findall('wfs:member/US_WBD_HUC_WBD:HUC12_US/US_WBD_HUC_WBD:HUC12', ns)[0].text
    
    return dict(huc12 = huc12,
                huc10 = huc12[:-2],
                huc8 = huc12[:-4],
                huc6 = huc12[:-6],
                huc4 = huc12[:-8],
                huc2 = huc12[:-10],
               )

In [66]:
lat = 44.8004
lon = -122.6089

getHUC(lat, lon)


{'huc12': '170900050601',
 'huc10': '1709000506',
 'huc8': '17090005',
 'huc6': '170900',
 'huc4': '1709',
 'huc2': '17'}